<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/2_Transformaciones_basicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proceso completo de Extracción, Transformación y Carga
Partimos del fichero con datos en bruto llamado **empleados_metacortex.csv**

In [ ]:
!git clone https://github.com/franciscogarate/cdiae

In [ ]:
import pandas as pd
file = 'cdiae/data/01_raw/empleados_metacortex.csv'
df = pd.read_csv(
        file,
        sep=';',                    # Separador de campos
        header=3,                   # Fila 4 contiene los encabezados (0-indexed, salta comentarios)
        encoding='utf-8',           # Codificación de caracteres
        skipfooter=1,               # Omite la última 1 fila (comentarios finales)
        engine='python',            # Necesario para skipfooter
        comment='%',                # Líneas que empiecen con % son comentarios
        usecols=['fecha_nacimiento','fecha_alta','nombre','sexo_biologico','nif','codigo_postal','genero','departamento','activo','horas_semanales','bonus'],
        parse_dates=['fecha_nacimiento', 'fecha_alta'],  # Convierte automáticamente a datetime
        date_format='%Y-%m-%d',      # OJO: Formato único para todas las columnas de fecha
        na_values=['', 'N/A', 'null', 'NULL'],
        keep_default_na=True,
)
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], format='%Y/%m/%d')
df

### Cruce de datos con otros ficheros
Partiendo de nuestros datos, vamos a complementarlos con información externa como los pagos efectuados que residen en un fichero de Excel externo.
Para ello utilizaremos las siguientes funciones de pandas:
  - `DataFrame.groupby`
  - `DataFrame.reset_index`
  - `pd.merge`

Lectura del Excel con pagos de empleados:

In [ ]:
df_pagos = pd.read_excel('cdiae/data/01_raw/pagos_empleados_metacortex.xlsx')
df_pagos.head()

Renombramos columnas para coincidir con el nombre de las columnas de 'df' antes del cruce

In [ ]:
df_pagos.rename(columns={'NIF': 'nif', 'IMPORTE': 'pago'}, inplace=True)
df_pagos.head()

In [ ]:
df_pagos.nif.value_counts()

Inspeccionamos los datos por su tipo, aquellos nulos y la memoria del DataFrame de pagos

In [ ]:
df_pagos.info()

Agregamos pagos por empleado (nif) sumando todos sus registros

In [ ]:
df_pagos_agg = df_pagos.groupby('nif')['pago'].sum().reset_index()
df_pagos_agg

Hacemos un left join para añadir 'pago' al DataFrame principal

In [ ]:
df = pd.merge(df, df_pagos_agg, on=['nif'], how='left')
df

## Distintos procesos de transformación de datos
### Map y replace
Unas de las funciones que podemos utilizar para reemplazar y unificar los valores posibles son `map` y `replace`.
Adicionalmente, para poder todos los valores que posee un campo podemos usar las funciones `unique` y `nunique`.

In [ ]:
df.sexo_biologico.unique()

Valores únicos de la columna 'genero' antes del mapeo

In [ ]:
df.genero.unique()

Homogeneizamos etiquetas: 'Mujer'->'F' y 'Hombre'->'M'

In [ ]:
df['genero'] = df['genero'].map({'Mujer':'F', 'Hombre':'M'})
df.genero.unique()                                                  # Valores únicos tras el mapeo

In [ ]:
df.activo.unique()                                                  # Valores únicos actuales de 'activo'

### Reemplazar con un condicional

Ejemplo de conversión con apply (mantener comentado si no se necesita)

In [ ]:
df['activo'] = df.apply(lambda x: True if x.activo == 'Sí' else False, axis=1)
df

Resumen de tipos de datos y memoria tras las transformaciones:

In [ ]:
df.info()

## Guardado de datos
- El último paso del proceso de ETL es la carga o guardado (loading). Para ello, no vamos a utilizar el mismo formato del fichero origen (raw) sino que dicho fichero transformado, siguiendo los estandares de buenas prácticas, vamos a guardarlo como ficheros intermedios en un formato que incorporé el formato.
- La mejor opción para ficheros intermedios son los formatos feather y parquet.
- Veamos las principales caracteristicas, así como pros y contras de cada uno de ellos.
### Feather
El formato Feather es un formato de archivo binario para almacenar datos de forma eficiente. Es un formato abierto y ligero que se utiliza para el intercambio de datos entre aplicaciones y lenguajes de programación.
Así, es el lenguage ideal para guardar nuestras base de datos _intermedias_ en nuestro flujo de transformaciones en python, así como para compartir dichos ficheros con otros programas escritos en R, ya que R tambien lee sin problemas el formato Feather.
### Parquet
El formato Parquet, en cambio, es un formato de archivo columnar (en columnas) muy eficiente que se utiliza para almacenar datos principalmente de texto. Tambien es un formato abierto y ligero que se utiliza para el intercambio de datos entre aplicaciones y lenguajes de programación.
- Ambos tienen compatibilidad con pandas, pero Parquet es más eficiente para grandes volúmenes de datos.
    - `df.to_parquet('ruta/archivo.parquet')`
    - `df.to_feather('ruta/archivo.feather')`
- Feather y Parquet mantienen los tipos de datos originales (int64, float64, datetime, etc.), a diferencia de CSV que convierte todo a texto.
- Las principales diferencias son:
    - Feather está diseñado específicamente para ser extremadamente rápido en lectura/escritura
    - Feather usa un formato binario sencillo basado en Apache Arrow, lo que lo hace ideal para intercambio rápido entre Python y R.
    - Parquet utiliza una compresión mas compleja pensada en reducir el tamaño
    - Feather tiene menor compresión que Parquet, resultando en archivos más grandes, pero con acceso más rápido.
- Existe tambien el formato pickle aunque es especifico para python
- Mi recomendación para ficheros temporales de cualquier tamaño es Feather, y Parquet para ficheros muy pesados que vayan a almacenarse en la nube (> 10 GB) o dataset con muchas columnas, ya que en estos casos, Parquet puede llegar a comprimir hasta 10 veces.
- Cuando trabajas con fichero .parquet no puedes importar las x primeras lineas.

In [ ]:
df.to_feather('cdiae/data/02_intermediate/empleados_metacortex.ftr')   # Guardamos el dataset intermedio en formato Feather